---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [26]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [27]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [28]:
def answer_one():
    
    total_msgs_count = len(spam_data)
    spam_msgs_count = len(spam_data[spam_data['target']==1])
    return (spam_msgs_count/total_msgs_count)*100

In [29]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    cv = CountVectorizer()
    cv.fit(X_train)
    vocab = cv.vocabulary_
    words = sorted(vocab.keys(), key =len)
    longest = words[-1]
    return longest

In [31]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [32]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    cv = CountVectorizer()
    cv_train = cv.fit_transform(X_train)
    multiNB = MultinomialNB(alpha=0.1)
    multiNB.fit(cv_train, y_train)
    cv_test = cv.transform(X_test)
    prediction = multiNB.predict(cv_test)
    score = roc_auc_score(y_test, prediction)
    return score

In [33]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    tfidf_vect = TfidfVectorizer()
    tfidf_train = tfidf_vect.fit_transform(X_train)
    features = tfidf_vect.get_feature_names()
    indices = np.argsort(tfidf_vect.idf_)[::-1]
    top_features = [(features[i],tfidf_vect.idf_[i]) for i in indices[:20]]
    bottom_features = [(features[i],tfidf_vect.idf_[i]) for i in indices[-20:]]
    top_list = list(zip(*top_features))
    top_series = pd.Series(top_list[1], index=top_list[0])
    bottom_list = list(zip(*bottom_features))
    bottom_series = pd.Series(bottom_list[1], index=bottom_list[0])
    top_series.sort_values(ascending=False, inplace=True)
    bottom_series.sort_values(ascending=True, inplace=True)
    
    return (bottom_series, top_series)

In [35]:
answer_four()

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64, nickey         8.644919
 nicky          8.644919
 companies      8.644919
 nmde           8.644919
 nitz           8.644919
 nitros         8.644919
 nitro          8.644919
 comb           8.644919
 combination    8.644919
 nino           8.644919
 nimbomsons     8.644919
 nikiyu4        8.644919
 nike           8.644919
 nigro          8.644919
 nightnight     8.644919
 nighters       8.644919
 comfey         8.644919
 nigh           8.644919
 comfort        8.644919
 ûówell         8.644919
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [36]:
def answer_five():
    tfidf_vect = TfidfVectorizer(min_df=3)
    tfidf_train = tfidf_vect.fit_transform(X_train)
    multiNB = MultinomialNB(alpha=0.1)
    multiNB.fit(tfidf_train, y_train)
    tfidf_test = tfidf_vect.transform(X_test)
    prediction = multiNB.predict(tfidf_test)
    score = roc_auc_score(y_test, prediction)
    return score

In [37]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [38]:
def answer_six():
    spam_df = spam_data[spam_data['target']==1]
    not_spam_df = spam_data[spam_data['target']==0]
    spam_df['Words_Count'] = spam_df['text'].apply(lambda x: len(x))
    not_spam_df['Words_Count'] = not_spam_df['text'].apply(lambda x: len(x))
    spam_avg = (spam_df['Words_Count'].sum()/len(spam_df))
    not_spam_avg = (not_spam_df['Words_Count'].sum()/len(not_spam_df))
    return (not_spam_avg, spam_avg)

In [39]:
answer_six()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [40]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [41]:
from sklearn.svm import SVC

def answer_seven():
    X_train_chars = X_train.apply(lambda x: len(x))
    X_test_chars = X_test.apply(lambda x: len(x))
    tfidf_vect = TfidfVectorizer(min_df=5)
    tfidf_train = tfidf_vect.fit_transform(X_train)
    x_train_updated = add_feature(tfidf_train, [X_train_chars])
    tfidf_test = tfidf_vect.transform(X_test)
    x_test_updated = add_feature(tfidf_test, [X_test_chars])
    classifier = SVC(C=10000)
    classifier.fit(x_train_updated, y_train)
    prediction = classifier.predict(x_test_updated)
    score = roc_auc_score(y_test, prediction)
    return score

In [42]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [43]:
def answer_eight():
    spam_df = spam_data[spam_data['target']==1]
    not_spam_df = spam_data[spam_data['target']==0]
    spam_df['digits_count'] = spam_df['text'].apply(lambda x: sum(c.isdigit() for c in x))
    not_spam_df['digits_count'] = not_spam_df['text'].apply(lambda x: sum(c.isdigit() for c in x))
    spam_avg = (spam_df['digits_count'].sum()/len(spam_df))
    not_spam_avg = (not_spam_df['digits_count'].sum()/len(not_spam_df))
    return (not_spam_avg, spam_avg)

In [44]:
answer_eight()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [45]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    X_train_chars = X_train.apply(lambda x: len(x))
    X_test_chars = X_test.apply(lambda x: len(x))
    X_train_digits = X_train.apply(lambda x: sum(c.isdigit() for c in x))
    X_test_digits = X_test.apply(lambda x: sum(c.isdigit() for c in x))
    
    tfidf_vect = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
    tfidf_train = tfidf_vect.fit_transform(X_train)
    X_train_updated = add_feature(tfidf_train, [X_train_chars, X_train_digits])
    
    tfidf_test = tfidf_vect.transform(X_test)
    X_test_updated = add_feature(tfidf_test, [X_test_chars, X_test_digits])
    
    lr = LogisticRegression(C=100)
    lr.fit(X_train_updated, y_train)
    prediction = lr.predict(X_test_updated)
    score = roc_auc_score(y_test, prediction)
    
    return score

In [46]:
answer_nine()

0.96787090640544626

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [47]:
def answer_ten():
    spam_df = spam_data[spam_data['target']==1]
    not_spam_df = spam_data[spam_data['target']==0]
    
    spam_df['non_word'] = spam_df['text'].str.findall(r'\W')
    not_spam_df['non_word'] = not_spam_df['text'].str.findall(r'\W')
    
    spam_df['non_word_count'] = spam_df['non_word'].apply(lambda x: len(x))
    not_spam_df['non_word_count'] = not_spam_df['non_word'].apply(lambda x: len(x))
    
    spam_avg = (spam_df['non_word_count'].sum()/len(spam_df))
    not_spam_avg = (not_spam_df['non_word_count'].sum()/len(not_spam_df))
    
    return (not_spam_avg, spam_avg)

In [48]:
answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [61]:
def answer_eleven():
    cv = CountVectorizer(min_df = 5, ngram_range = (2, 5), analyzer='char_wb')
    cv_train = cv.fit_transform(X_train)
    
    X_train_chars = X_train.apply(lambda x: len(x))
    X_test_chars = X_test.apply(lambda x: len(x))
    
    X_train_digits = X_train.apply(lambda x: sum(c.isdigit() for c in x))
    X_test_digits = X_test.apply(lambda x: sum(c.isdigit() for c in x))
    
    X_train_non_word = X_train.str.findall(r'\W')
    X_train_non_word_len = X_train_non_word.apply(lambda x: len(x))
    
    X_test_non_word = X_test.str.findall(r'\W')
    X_test_non_word_len = X_test_non_word.apply(lambda x: len(x))
    
    X_train_updated = add_feature(cv_train, [X_train_chars, X_train_digits, X_train_non_word_len])
    
    cv_test = cv.transform(X_test)
    X_test_updated = add_feature(cv_test, [X_test_chars, X_test_digits, X_test_non_word_len])
    
    lr = LogisticRegression(C=100)
    lr.fit(X_train_updated, y_train)
    prediction = lr.predict(X_test_updated)
    auc_score = roc_auc_score(prediction, y_test)
    feature_names = np.array(cv.get_feature_names())
    feature_names = np.append(feature_names, ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coeff_index = lr.coef_[0].argsort()
    smallest_coeff = feature_names[sorted_coeff_index[:10]]
    largest_coeff = feature_names[sorted_coeff_index[:-11:-1]]
    
    return (auc_score, smallest_coeff, largest_coeff)

In [62]:
answer_eleven()

(0.9914366108841286,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U19'),
 array(['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww',
        'ar'], 
       dtype='<U19'))